# Project_AI_AGH

## Import data

### Import libraries. Using keras to build model with backend tensorflow

In [1]:
import pandas
import dateutil
import datetime
import matplotlib.pyplot as plt
import os
from os import listdir
from os.path import isfile, join
import numpy
from numpy import ndarray
import sys
import keras
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

C:\Users\minhn\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Read Data

#### Read all images_params

In [2]:
images_params = pandas.read_csv("NAPS_valence_arousal_2014.csv", sep=";", header=0, decimal=",")

In [3]:
#Check images_parasm (NAPS****.csv)
images_params

,ID,Category,Nr,V_H,Description,Valence,"Arousal,,"
0,Animals_001_h,Animals,1,h,Dead Stork,2.57,6.44
1,Animals_002_v,Animals,2,v,Lion,6.24,6.68
2,Animals_003_h,Animals,3,h,Snake,5.24,5.52
3,Animals_004_v,Animals,4,v,Wolf,4.50,7.20
4,Animals_005_h,Animals,5,h,Bat,5.31,5.82
5,Animals_006_v,Animals,6,v,Snake,5.13,6.23
6,Animals_007_h,Animals,7,h,Wolf,4.76,7.60
7,Animals_008_v,Animals,8,v,Fighting Chickens,2.63,6.80
8,Animals_009_v,Animals,9,v,Cat,5.79,5.61
9,Animals_010_h,Animals,10,h,Sick Kitten,4.59,5.90


#### Define necessary functions for timeStamp and to clear prefix, suffix of file name

In [4]:
#Provided by example
def convert_psychopy_timestamp(timestamp_string):
    dt = dateutil.parser.parse(timestamp_string)  # curr time
    epoch = datetime.datetime.utcfromtimestamp(0) # timestamp 0
    ts = (dt - epoch).total_seconds() * 1000.0    # get time difference in milliseconds
    return ts

#Provided by example
def clear_filename(filename):
    start_offset = len("pictures\\day1\\")
    end_offset = len(".jpg")
    return filename[start_offset:][:-end_offset]


#### Define functions which find all csv file in a directory, function which finds all directories, ...

In [5]:
#Finding all csv file following the path of provided directory
def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

#Finding all subdirectories following the path of provided directory
def find_subdirectories( path_to_dir,):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if not isfile(join(path_to_dir, filename)) ]

#Finding in my_list existing a file which its name starts with my_pattern
def find_list_string_startsWith (my_list, my_pattern):
    return [x for x in my_list if x.startswith(my_pattern)]


#### Parameters for reading file, declaring all dictionaries

In [6]:
path_to_all_participants = "raw-data\\2018-afcai-spring\\"
list_all_participants=find_subdirectories(path_to_all_participants)
list_all_participants=[person for person in list_all_participants if person != 'C309']

#all dictionaries which will be used during processing data
#key: research participant. ex: B303, C309
my_dictionay_images_params = {}
my_dictionary_images_timestamp = {}
my_dictionary_HR_mV = {}
my_dictionary_HR_bps = {}
my_dictionary_HR_nW = {}
my_dictionary_GSR = {}

In [7]:
#find GSR for a person and convert all into miliSiemens
def find_GSR(path):
    list_subDirectories = find_subdirectories(path)
    if 'BITalino' in list_subDirectories:
        csv_files = find_csv_filenames(path + 'BITalino\\')  
        csv_file_GSR = find_list_string_startsWith(csv_files, 'GSR')
        return pandas.read_csv(path + 'BITalino\\' + csv_file_GSR[0], sep=",", names=["Timestamp", "Value"], skiprows=1)
    elif ('Empatica' in  list_subDirectories) and ('eHealth' in list_subDirectories): 
        csv_files = find_csv_filenames(path + 'eHealth\\')
        csv_file_BVP = find_list_string_startsWith(csv_files, 'GSR')
        data = pandas.read_csv(path + 'eHealth\\'  + csv_file_BVP[0], sep=",", names=["Timestamp", "Value"], skiprows=1)
        data.Value = data.Value.apply(func=lambda t: 1000000*(1/t))
        return data
    else:
        subdirectory = list_subDirectories[0]
        csv_files = find_csv_filenames(path + subdirectory + '\\')
        if (subdirectory == 'Empatica'): 
            csv_file_GSR = find_list_string_startsWith(csv_files, 'GSR')
            return pandas.read_csv(path + subdirectory + '\\' + csv_file_GSR[0], sep=";", names=["Timestamp", "Value"])
        else:
            csv_file_GSR = find_list_string_startsWith(csv_files, 'GSR')
            data = pandas.read_csv(path + subdirectory + '\\' + csv_file_GSR[0], sep=",", names=["Timestamp", "Value"], skiprows=1)
            data.Value = data.Value.apply(func=lambda t: 1000000*(1/t))
            return data

#Finding HR and fill it to correspondent dictionary
def fill_HR(person, path):
    list_subDirectories = find_subdirectories(path)
    if 'BITalino' in list_subDirectories:
        csv_files = find_csv_filenames(path + 'BITalino\\')
        
        csv_file_BPM = find_list_string_startsWith(csv_files, 'BPM')
        my_dictionary_HR_mV[person] = pandas.read_csv(path + 'BITalino\\'  + csv_file_BPM[0], sep=",", names=["Timestamp", "Value"], skiprows=1)
        my_dictionary_HR_bps[person] = pandas.DataFrame({'Timestamp': my_dictionary_HR_mV[person].Timestamp, 'Value': 0})
        my_dictionary_HR_nW[person] = pandas.DataFrame({'Timestamp': my_dictionary_HR_mV[person].Timestamp, 'Value': 0})
        
    elif ('Empatica' in  list_subDirectories) and ('eHealth' in list_subDirectories): 
        csv_files = find_csv_filenames(path + 'Empatica\\')
        csv_file_BVP = find_list_string_startsWith(csv_files, 'BVP')
        my_dictionary_HR_nW[person] = pandas.read_csv(path + 'Empatica\\'  + csv_file_BVP[0], sep=";", names=["Timestamp", "Value"])
        my_dictionary_HR_mV[person] = pandas.DataFrame({'Timestamp': my_dictionary_HR_nW[person].Timestamp, 'Value': 0})
        my_dictionary_HR_bps[person] = pandas.DataFrame({'Timestamp': my_dictionary_HR_nW[person].Timestamp, 'Value': 0})
    
    else:
        subdirectory = list_subDirectories[0]
        csv_files = find_csv_filenames(path + subdirectory + '\\')
        if (subdirectory == 'Empatica'):
            csv_file_BVP = find_list_string_startsWith(csv_files, 'BVP')
            my_dictionary_HR_nW[person] = pandas.read_csv(path + subdirectory + '\\'  + csv_file_BVP[0], sep=";", names=["Timestamp", "Value"])
            my_dictionary_HR_mV[person] = pandas.DataFrame({'Timestamp': my_dictionary_HR_nW[person].Timestamp, 'Value': 0})
            my_dictionary_HR_bps[person] = pandas.DataFrame({'Timestamp': my_dictionary_HR_nW[person].Timestamp, 'Value': 0})

        else:
            csv_file_BPM = find_list_string_startsWith(csv_files, 'BPM')
            my_dictionary_HR_bps[person] = pandas.read_csv(path + subdirectory + '\\'  + csv_file_BPM[0], sep=",", names=["Timestamp", "Value"], skiprows=1)
            my_dictionary_HR_mV[person] = pandas.DataFrame({'Timestamp': my_dictionary_HR_bps[person].Timestamp, 'Value': 0})
            my_dictionary_HR_nW[person] = pandas.DataFrame({'Timestamp': my_dictionary_HR_bps[person].Timestamp, 'Value': 0})

#### Finding all necessary dictionaries

In [8]:
for research_participant in list_all_participants:
    path = path_to_all_participants + research_participant + '\\'
    csv_file = find_csv_filenames(path)
    my_dictionary_images_timestamp[research_participant] = pandas.read_csv(path + csv_file[0], sep=",", names=["Image", "Timestamp"], skiprows=6)
    
for research_participant in list_all_participants:
    path = path_to_all_participants + research_participant + '\\'
    fill_HR(research_participant, path)
    my_dictionary_GSR[research_participant] = find_GSR(path)

### Preprocessing and converting

#### Define converting function

In [9]:
#Provied by example
def convert_timestamp_dictionary(my_dictionary):
    for key, value in my_dictionary.items():
        if (any(e < 10 ** 11 for e in value.Timestamp)):
            my_dictionary[key].Timestamp = my_dictionary[key].Timestamp.apply(func=lambda t: 1000.0 * (t + 60*60))
        else:
            my_dictionary[key].Timestamp = my_dictionary[key].Timestamp.apply(func=lambda t: t + 60*60*1000)
    return my_dictionary

#### Process dictionary HR and images_timestamp which main purpose is converting timestamp

In [10]:
for key, value in my_dictionary_images_timestamp.items():
    my_dictionary_images_timestamp[key].Image = my_dictionary_images_timestamp[key].Image.apply(func=clear_filename)
    my_dictionary_images_timestamp[key].Timestamp = my_dictionary_images_timestamp[key].Timestamp.apply(func=convert_psychopy_timestamp) 

my_dictionary_HR_mV = convert_timestamp_dictionary(my_dictionary_HR_mV)
my_dictionary_HR_nW = convert_timestamp_dictionary(my_dictionary_HR_nW)
my_dictionary_HR_bps = convert_timestamp_dictionary(my_dictionary_HR_bps)
my_dictionary_GSR = convert_timestamp_dictionary(my_dictionary_GSR)

#### Processing and converting images in Naps to dict with id = image name

In [11]:
my_dictionay_images_params = images_params.set_index('ID').T.to_dict('list')

In [12]:
#check dictionary_images_timestamp
my_dictionary_images_timestamp['B303']

,Image,Timestamp
0,Landscapes_097_v,1.520850e+12
1,Landscapes_049_h,1.520850e+12
2,Objects_160_h,1.520850e+12
3,Animals_056_h,1.520850e+12
4,People_038_h,1.520850e+12
5,People_130_h,1.520850e+12
6,Landscapes_050_h,1.520850e+12
7,Landscapes_104_h,1.520850e+12
8,Animals_020_h,1.520850e+12
9,Landscapes_067_h,1.520850e+12


### Analysis

#### Finding dataset from raw_data

In [13]:
#Finding mean of a provided DataFrame as argument which Timestamp in range (start_time, end_time) 
def get_mean(dataframe, start_time, end_time):
    selected_rows = dataframe.loc[(dataframe['Timestamp'] >= start_time) & (dataframe['Timestamp'] < end_time)]
    list_value = []
    for reading in selected_rows.itertuples():
        list_value.append(reading.Value)
    if not list_value:
        return 0
    else:
        return numpy.mean(list_value)

In [14]:
#define data_set like a list
data_set = list()
#for each person in list of participants
for person in list_all_participants:
    #get data of image and timestamp starting this image
    images_timestamp = my_dictionary_images_timestamp[person]
    for image_index in range(len(images_timestamp.Image)):
        #get image id, valence, arousal
        image = images_timestamp.Image[image_index]
        valence = my_dictionay_images_params[image][4]
        arousal = my_dictionay_images_params[image][5]
        #get start_time
        start_time = images_timestamp.Timestamp[image_index]
        #get end_tie
        end_time = images_timestamp.Timestamp[image_index + 1] if image_index != len(images_timestamp.Image) - 1 else sys.float_info.max
        
        #calculating all means
        mean_mV = get_mean(my_dictionary_HR_mV[person], start_time, end_time)
        mean_nW = get_mean(my_dictionary_HR_nW[person], start_time, end_time)
        mean_bps = get_mean(my_dictionary_HR_bps[person], start_time, end_time)
        mean_GSR = get_mean(my_dictionary_GSR[person], start_time, end_time)
        
        #add data into data_set as a tuple with (Image_id, mean of mV, mean of bps, mean of GSR, valence, arousal)
        data = (image, mean_mV, mean_nW, mean_bps, mean_GSR, valence, arousal)
        data_set.append(data)

#### Check data set

In [15]:
#We have 97 people and for each people we have 42 - 6 trail images to watch
#So in totally we have 97 * 36 = 3492
len(data_set)

3492

In [16]:
#Write data to a file which can be usefull in the future
with open('data.csv', 'w') as fp:
    fp.write("Image, mV, nW, bps, GSR, valence, arousal\n")
    fp.write('\n'.join('%s,%s,%s,%s,%s,%s,%s' %x for x in data_set))

### Building model

#### Divide raw data set into training set and test set

In [17]:
#We decided 8:2 for training:test because size of data_set is only 3500
#It could be 7:3 for second model
n_dataset = len(data_set)
n_training = int (n_dataset / 100 * 80.0)
n_test = n_dataset - n_training

In [18]:
#get raw training data, raw test data
raw_training = data_set[: n_training]
raw_test = data_set[n_training:]

In [19]:
#pass a list into array of numpy for easier processing
np_array_training = numpy.asarray(raw_training)
#delte image name, valence and arousal and convert type to float
X_training = numpy.delete(np_array_training, [0, 5, 6], 1)
X_training = X_training.astype(float)

#hold only valence and convert type to uint8
Y_training_valence = numpy.delete(np_array_training, [0, 1, 2, 3, 4, 6], 1)
Y_training_valence = ndarray.flatten(Y_training_valence)
Y_training_valence = Y_training_valence.astype(float)
Y_training_valence = Y_training_valence.astype(int)

#hold only arousal and convert type to uint8
Y_training_arousal = numpy.delete(np_array_training, [0, 1, 2, 3, 4, 5], 1)
Y_training_arousal = ndarray.flatten(Y_training_arousal)
Y_training_arousal = Y_training_arousal.astype(float)
Y_training_arousal = Y_training_arousal.astype(int)

#similar to test set
np_array_test = numpy.asarray(raw_test)
X_test = numpy.delete(np_array_test, [0, 5, 6], 1)
X_test = X_test.astype(float)

Y_test_valence = numpy.delete(np_array_test, [0, 1, 2, 3, 4, 6], 1)
Y_test_valence = ndarray.flatten(Y_test_valence)
Y_test_valence = Y_test_valence.astype(float)
Y_test_valence = Y_test_valence.astype(int)

Y_test_arousal = numpy.delete(np_array_test, [0, 1, 2, 3, 4, 5], 1)
Y_test_arousal = ndarray.flatten(Y_test_arousal)
Y_test_arousal = Y_test_arousal.astype(float)
Y_test_valence = Y_test_valence.astype(int)

#### Checking shape of data

In [20]:
#We expected to have (2794, 4) for training and (699, 4) for testing
print(X_training.shape)
print(Y_training_valence.shape)
print(Y_training_arousal.shape)
print(X_test.shape)
print(Y_test_valence.shape)
print(Y_test_arousal.shape)

(2793, 4)
(2793,)
(2793,)
(699, 4)
(699,)
(699,)


#### Conver to softmax layer

In [21]:
#For the purpose of neutron network. We always decided to get one hot on coding
#For example we have level of valence = 2, the neutron network will see it as 0 0 1 0 0 0 0 0 0 0
#We convert it by categorical in keras
n_classes = 10
Y_training_valence = keras.utils.to_categorical(Y_training_valence, n_classes)
Y_training_arousal = keras.utils.to_categorical(Y_training_arousal, n_classes)
Y_test_valence = keras.utils.to_categorical(Y_test_valence, n_classes)
Y_test_arousal = keras.utils.to_categorical(Y_test_arousal, n_classes)

#### Design neutral network

In [22]:
#Design neutral network architecture with only one Dense layer
#Activation softmax means output 
model_valence = Sequential()
model_valence.add(Dense(64, activation = 'sigmoid', input_shape = (4,)))
model_valence.add(Dense(10, activation='softmax'))

model_arousal = Sequential()
model_arousal.add(Dense(64, activation = 'sigmoid', input_shape = (4,)))
model_arousal.add(Dense(10, activation='softmax'))

In [23]:
#We summary the network architecture
#Expectation for param: first dense layer: 64*4+64(input) and second dense layer: 64*10+10(output)
model_valence.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                320       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 970
Trainable params: 970
Non-trainable params: 0
_________________________________________________________________


In [24]:
#We summary the network architecture
#Expectation for param: first dense layer: 64*4+64(input) and second dense layer: 64*10+10(output)
model_arousal.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                320       
_________________________________________________________________
dense_4 (Dense)              (None, 10)                650       
Total params: 970
Trainable params: 970
Non-trainable params: 0
_________________________________________________________________


#### Configuration model

In [25]:
#we use loss = mean squared error
#gradient descent with learning rate= 0.01
#optimize with accuracy
model_valence.compile(loss='mean_squared_error', optimizer=SGD(lr=0.01), metrics=['accuracy'])
model_arousal.compile(loss='mean_squared_error', optimizer=SGD(lr=0.01), metrics=['accuracy'])

#### Train

In [26]:
#train model with training set, 100 epochs(100 times), verbose = 1 means we will see result after one epochs
#validation by test_set
model_valence.fit(X_training, Y_training_valence, epochs = 100, verbose = 1, validation_data=(X_test, Y_test_valence))

Train on 2793 samples, validate on 699 samples
Epoch 1/100
2793/2793 [==============================] - 0s 169us/step - loss: 0.0988 - acc: 0.1149 - val_loss: 0.0960 - val_acc: 0.1245
Epoch 2/100
2793/2793 [==============================] - 0s 45us/step - loss: 0.0972 - acc: 0.1174 - val_loss: 0.0946 - val_acc: 0.1302
Epoch 3/100
2793/2793 [==============================] - 0s 47us/step - loss: 0.0959 - acc: 0.1174 - val_loss: 0.0936 - val_acc: 0.1330
Epoch 4/100
2793/2793 [==============================] - 0s 40us/step - loss: 0.0948 - acc: 0.1185 - val_loss: 0.0927 - val_acc: 0.1330
Epoch 5/100
2793/2793 [==============================] - 0s 40us/step - loss: 0.0939 - acc: 0.1189 - val_loss: 0.0919 - val_acc: 0.1316
Epoch 6/100
2793/2793 [==============================] - 0s 44us/step - loss: 0.0931 - acc: 0.1214 - val_loss: 0.0912 - val_acc: 0.1345
Epoch 7/100
2793/2793 [==============================] - 0s 58us/step - loss: 0.0923 - acc: 0.1271 - val_loss: 0.0905 - val_acc: 0.1330


2793/2793 [==============================] - 0s 40us/step - loss: 0.0829 - acc: 0.2470 - val_loss: 0.0833 - val_acc: 0.2260
Epoch 61/100
2793/2793 [==============================] - 0s 38us/step - loss: 0.0829 - acc: 0.2470 - val_loss: 0.0833 - val_acc: 0.2260
Epoch 62/100
2793/2793 [==============================] - 0s 34us/step - loss: 0.0829 - acc: 0.2470 - val_loss: 0.0833 - val_acc: 0.2260
Epoch 63/100
2793/2793 [==============================] - 0s 44us/step - loss: 0.0829 - acc: 0.2463 - val_loss: 0.0833 - val_acc: 0.2289
Epoch 64/100
2793/2793 [==============================] - 0s 31us/step - loss: 0.0829 - acc: 0.2470 - val_loss: 0.0833 - val_acc: 0.2275
Epoch 65/100
2793/2793 [==============================] - 0s 34us/step - loss: 0.0829 - acc: 0.2460 - val_loss: 0.0833 - val_acc: 0.2260
Epoch 66/100
2793/2793 [==============================] - 0s 41us/step - loss: 0.0829 - acc: 0.2478 - val_loss: 0.0833 - val_acc: 0.2289
Epoch 67/100
2793/2793 [==============================

In [27]:
model_arousal.fit(X_training, Y_training_arousal, epochs = 100, verbose = 1, validation_data=(X_test, Y_test_arousal))

Train on 2793 samples, validate on 699 samples
Epoch 1/100
2793/2793 [==============================] - 0s 73us/step - loss: 0.0949 - acc: 0.0616 - val_loss: 0.0951 - val_acc: 0.1102
Epoch 2/100
2793/2793 [==============================] - 0s 39us/step - loss: 0.0937 - acc: 0.1214 - val_loss: 0.0943 - val_acc: 0.1359
Epoch 3/100
2793/2793 [==============================] - 0s 47us/step - loss: 0.0929 - acc: 0.1389 - val_loss: 0.0937 - val_acc: 0.1373
Epoch 4/100
2793/2793 [==============================] - 0s 43us/step - loss: 0.0922 - acc: 0.1686 - val_loss: 0.0931 - val_acc: 0.1602
Epoch 5/100
2793/2793 [==============================] - 0s 52us/step - loss: 0.0916 - acc: 0.1969 - val_loss: 0.0925 - val_acc: 0.1674
Epoch 6/100
2793/2793 [==============================] - 0s 50us/step - loss: 0.0910 - acc: 0.2019 - val_loss: 0.0920 - val_acc: 0.1688
Epoch 7/100
2793/2793 [==============================] - 0s 50us/step - loss: 0.0904 - acc: 0.2026 - val_loss: 0.0915 - val_acc: 0.1731
E

2793/2793 [==============================] - 0s 48us/step - loss: 0.0812 - acc: 0.2703 - val_loss: 0.0815 - val_acc: 0.2675
Epoch 61/100
2793/2793 [==============================] - 0s 35us/step - loss: 0.0812 - acc: 0.2692 - val_loss: 0.0815 - val_acc: 0.2704
Epoch 62/100
2793/2793 [==============================] - 0s 39us/step - loss: 0.0812 - acc: 0.2700 - val_loss: 0.0814 - val_acc: 0.2704
Epoch 63/100
2793/2793 [==============================] - 0s 33us/step - loss: 0.0811 - acc: 0.2700 - val_loss: 0.0814 - val_acc: 0.2704
Epoch 64/100
2793/2793 [==============================] - 0s 35us/step - loss: 0.0811 - acc: 0.2675 - val_loss: 0.0813 - val_acc: 0.2704
Epoch 65/100
2793/2793 [==============================] - 0s 30us/step - loss: 0.0811 - acc: 0.2689 - val_loss: 0.0813 - val_acc: 0.2690
Epoch 66/100
2793/2793 [==============================] - 0s 33us/step - loss: 0.0810 - acc: 0.2710 - val_loss: 0.0813 - val_acc: 0.2690
Epoch 67/100
2793/2793 [==============================

### Second model 

#### Design neutral network

In [28]:
#We add more layer into network and using activation tanh
model_valence = Sequential()
model_valence.add(Dense(64, activation = 'tanh', input_shape = (4,)))
model_valence.add(Dense(64, activation = 'tanh'))
model_valence.add(Dense(64, activation = 'tanh'))
model_valence.add(Dense(10, activation='softmax'))

model_arousal = Sequential()
model_arousal.add(Dense(64, activation = 'tanh', input_shape = (4,)))
model_arousal.add(Dense(64, activation = 'tanh'))
model_arousal.add(Dense(64, activation = 'tanh'))
model_arousal.add(Dense(10, activation='softmax'))

#### Configuration model

In [29]:
#We ise lower learning rate and loss = categorical_crossentropy
model_valence.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.001), metrics=['accuracy'])
model_arousal.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.001), metrics=['accuracy'])

#### Train

In [30]:
#train model with training set, 100 epochs(100 times), verbose = 1 means we will see result after one epochs
#validation by test_set
model_valence.fit(X_training, Y_training_valence, epochs = 100, verbose = 1, validation_data=(X_test, Y_test_valence))

Train on 2793 samples, validate on 699 samples
Epoch 1/100
2793/2793 [==============================] - 0s 141us/step - loss: 2.3439 - acc: 0.1507 - val_loss: 2.2195 - val_acc: 0.1617
Epoch 2/100
2793/2793 [==============================] - 0s 38us/step - loss: 2.1371 - acc: 0.2023 - val_loss: 2.0642 - val_acc: 0.2074
Epoch 3/100
2793/2793 [==============================] - 0s 46us/step - loss: 2.0397 - acc: 0.2324 - val_loss: 1.9942 - val_acc: 0.2217
Epoch 4/100
2793/2793 [==============================] - 0s 45us/step - loss: 1.9899 - acc: 0.2370 - val_loss: 1.9565 - val_acc: 0.2203
Epoch 5/100
2793/2793 [==============================] - 0s 43us/step - loss: 1.9606 - acc: 0.2356 - val_loss: 1.9345 - val_acc: 0.2203
Epoch 6/100
2793/2793 [==============================] - 0s 48us/step - loss: 1.9417 - acc: 0.2377 - val_loss: 1.9136 - val_acc: 0.2303
Epoch 7/100
2793/2793 [==============================] - 0s 49us/step - loss: 1.9264 - acc: 0.2385 - val_loss: 1.9010 - val_acc: 0.2289


2793/2793 [==============================] - 0s 43us/step - loss: 1.8453 - acc: 0.2521 - val_loss: 1.8252 - val_acc: 0.2546
Epoch 61/100
2793/2793 [==============================] - 0s 45us/step - loss: 1.8447 - acc: 0.2510 - val_loss: 1.8251 - val_acc: 0.2546
Epoch 62/100
2793/2793 [==============================] - 0s 39us/step - loss: 1.8446 - acc: 0.2521 - val_loss: 1.8252 - val_acc: 0.2561
Epoch 63/100
2793/2793 [==============================] - 0s 39us/step - loss: 1.8444 - acc: 0.2513 - val_loss: 1.8253 - val_acc: 0.2546
Epoch 64/100
2793/2793 [==============================] - 0s 36us/step - loss: 1.8439 - acc: 0.2521 - val_loss: 1.8261 - val_acc: 0.2532
Epoch 65/100
2793/2793 [==============================] - 0s 38us/step - loss: 1.8438 - acc: 0.2528 - val_loss: 1.8243 - val_acc: 0.2575
Epoch 66/100
2793/2793 [==============================] - 0s 40us/step - loss: 1.8437 - acc: 0.2506 - val_loss: 1.8270 - val_acc: 0.2575
Epoch 67/100
2793/2793 [==============================

In [31]:
model_arousal.fit(X_training, Y_training_arousal, epochs = 100, verbose = 1, validation_data=(X_test, Y_test_arousal))

Train on 2793 samples, validate on 699 samples
Epoch 1/100
2793/2793 [==============================] - 0s 116us/step - loss: 2.2862 - acc: 0.1697 - val_loss: 2.1282 - val_acc: 0.2561
Epoch 2/100
2793/2793 [==============================] - 0s 38us/step - loss: 2.0312 - acc: 0.2624 - val_loss: 1.9672 - val_acc: 0.2718
Epoch 3/100
2793/2793 [==============================] - 0s 52us/step - loss: 1.9275 - acc: 0.2610 - val_loss: 1.8995 - val_acc: 0.2647
Epoch 4/100
2793/2793 [==============================] - 0s 47us/step - loss: 1.8725 - acc: 0.2760 - val_loss: 1.8428 - val_acc: 0.2589
Epoch 5/100
2793/2793 [==============================] - 0s 50us/step - loss: 1.8403 - acc: 0.2760 - val_loss: 1.8149 - val_acc: 0.2718
Epoch 6/100
2793/2793 [==============================] - 0s 46us/step - loss: 1.8181 - acc: 0.2764 - val_loss: 1.7962 - val_acc: 0.2661
Epoch 7/100
2793/2793 [==============================] - 0s 40us/step - loss: 1.8016 - acc: 0.2725 - val_loss: 1.7772 - val_acc: 0.2518


2793/2793 [==============================] - 0s 44us/step - loss: 1.7007 - acc: 0.2757 - val_loss: 1.6706 - val_acc: 0.2761
Epoch 61/100
2793/2793 [==============================] - 0s 37us/step - loss: 1.7012 - acc: 0.2764 - val_loss: 1.6697 - val_acc: 0.2976
Epoch 62/100
2793/2793 [==============================] - 0s 41us/step - loss: 1.7003 - acc: 0.2764 - val_loss: 1.6708 - val_acc: 0.2876
Epoch 63/100
2793/2793 [==============================] - 0s 44us/step - loss: 1.6997 - acc: 0.2739 - val_loss: 1.6711 - val_acc: 0.2833
Epoch 64/100
2793/2793 [==============================] - 0s 37us/step - loss: 1.6996 - acc: 0.2775 - val_loss: 1.6688 - val_acc: 0.3076
Epoch 65/100
2793/2793 [==============================] - 0s 39us/step - loss: 1.6985 - acc: 0.2768 - val_loss: 1.6726 - val_acc: 0.2861
Epoch 66/100
2793/2793 [==============================] - 0s 39us/step - loss: 1.6985 - acc: 0.2764 - val_loss: 1.6703 - val_acc: 0.2833
Epoch 67/100
2793/2793 [==============================

### Third model

#### Design neutral network

In [32]:
#We add more layer into network and using activation tanh
#layer dense and BatchNormalization
model_valence = Sequential()
model_valence.add(Dense(64, activation = 'tanh', input_shape = (4,)))
model_valence.add(BatchNormalization())
model_valence.add(Dense(64, activation = 'tanh'))
model_valence.add(BatchNormalization())
model_valence.add(Dense(64, activation = 'tanh'))
model_valence.add(BatchNormalization())
model_valence.add(Dense(10, activation='softmax'))

model_arousal = Sequential()
model_arousal.add(Dense(64, activation = 'tanh', input_shape = (4,)))
model_arousal.add(BatchNormalization())
model_arousal.add(Dense(64, activation = 'tanh'))
model_arousal.add(BatchNormalization())
model_arousal.add(Dense(64, activation = 'tanh'))
model_arousal.add(BatchNormalization())
model_arousal.add(Dense(10, activation='softmax'))

#### Configuration model

In [33]:
#we use loss = mean squared error
#optimizer by adam which allows us to decrease learning rate overtime
#optimize with accuracy
model_valence.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model_arousal.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

#### Train

In [34]:
#train model with training set, 100 epochs(100 times), verbose = 1 means we will see result after one epochs
#validation by test_set
model_valence.fit(X_training, Y_training_valence, epochs = 100, verbose = 1, validation_data=(X_test, Y_test_valence))

Train on 2793 samples, validate on 699 samples
Epoch 1/100
2793/2793 [==============================] - 1s 497us/step - loss: 0.0906 - acc: 0.1880 - val_loss: 0.0879 - val_acc: 0.2318
Epoch 2/100
2793/2793 [==============================] - 0s 72us/step - loss: 0.0870 - acc: 0.2055 - val_loss: 0.0847 - val_acc: 0.2418
Epoch 3/100
2793/2793 [==============================] - 0s 77us/step - loss: 0.0853 - acc: 0.2245 - val_loss: 0.0858 - val_acc: 0.2160
Epoch 4/100
2793/2793 [==============================] - 0s 73us/step - loss: 0.0845 - acc: 0.2331 - val_loss: 0.0841 - val_acc: 0.2246
Epoch 5/100
2793/2793 [==============================] - 0s 67us/step - loss: 0.0842 - acc: 0.2159 - val_loss: 0.0841 - val_acc: 0.2189
Epoch 6/100
2793/2793 [==============================] - 0s 67us/step - loss: 0.0833 - acc: 0.2460 - val_loss: 0.0837 - val_acc: 0.2175
Epoch 7/100
2793/2793 [==============================] - 0s 71us/step - loss: 0.0831 - acc: 0.2345 - val_loss: 0.0832 - val_acc: 0.2375


2793/2793 [==============================] - 0s 72us/step - loss: 0.0824 - acc: 0.2427 - val_loss: 0.0829 - val_acc: 0.2461
Epoch 61/100
2793/2793 [==============================] - 0s 67us/step - loss: 0.0824 - acc: 0.2506 - val_loss: 0.0830 - val_acc: 0.2418
Epoch 62/100
2793/2793 [==============================] - 0s 75us/step - loss: 0.0824 - acc: 0.2420 - val_loss: 0.0830 - val_acc: 0.2346
Epoch 63/100
2793/2793 [==============================] - 0s 78us/step - loss: 0.0824 - acc: 0.2456 - val_loss: 0.0832 - val_acc: 0.2289
Epoch 64/100
2793/2793 [==============================] - 0s 75us/step - loss: 0.0825 - acc: 0.2410 - val_loss: 0.0829 - val_acc: 0.2418
Epoch 65/100
2793/2793 [==============================] - 0s 74us/step - loss: 0.0823 - acc: 0.2449 - val_loss: 0.0830 - val_acc: 0.2375
Epoch 66/100
2793/2793 [==============================] - 0s 69us/step - loss: 0.0824 - acc: 0.2481 - val_loss: 0.0831 - val_acc: 0.2446
Epoch 67/100
2793/2793 [==============================

In [35]:
model_arousal.fit(X_training, Y_training_arousal, epochs = 100, verbose = 1, validation_data=(X_test, Y_test_arousal))

Train on 2793 samples, validate on 699 samples
Epoch 1/100
2793/2793 [==============================] - 1s 466us/step - loss: 0.0908 - acc: 0.2213 - val_loss: 0.0880 - val_acc: 0.2017
Epoch 2/100
2793/2793 [==============================] - 0s 77us/step - loss: 0.0866 - acc: 0.2220 - val_loss: 0.0835 - val_acc: 0.2618
Epoch 3/100
2793/2793 [==============================] - 0s 76us/step - loss: 0.0839 - acc: 0.2496 - val_loss: 0.0829 - val_acc: 0.2504
Epoch 4/100
2793/2793 [==============================] - 0s 71us/step - loss: 0.0827 - acc: 0.2517 - val_loss: 0.0814 - val_acc: 0.2661
Epoch 5/100
2793/2793 [==============================] - 0s 80us/step - loss: 0.0822 - acc: 0.2549 - val_loss: 0.0810 - val_acc: 0.2933
Epoch 6/100
2793/2793 [==============================] - 0s 69us/step - loss: 0.0814 - acc: 0.2542 - val_loss: 0.0802 - val_acc: 0.2961
Epoch 7/100
2793/2793 [==============================] - 0s 78us/step - loss: 0.0806 - acc: 0.2621 - val_loss: 0.0803 - val_acc: 0.2747


2793/2793 [==============================] - 0s 72us/step - loss: 0.0796 - acc: 0.2649 - val_loss: 0.0806 - val_acc: 0.2704
Epoch 61/100
2793/2793 [==============================] - 0s 79us/step - loss: 0.0796 - acc: 0.2678 - val_loss: 0.0802 - val_acc: 0.2532
Epoch 62/100
2793/2793 [==============================] - 0s 77us/step - loss: 0.0795 - acc: 0.2610 - val_loss: 0.0807 - val_acc: 0.2632
Epoch 63/100
2793/2793 [==============================] - 0s 80us/step - loss: 0.0797 - acc: 0.2760 - val_loss: 0.0803 - val_acc: 0.2704
Epoch 64/100
2793/2793 [==============================] - 0s 105us/step - loss: 0.0794 - acc: 0.2782 - val_loss: 0.0810 - val_acc: 0.2504
Epoch 65/100
2793/2793 [==============================] - 0s 96us/step - loss: 0.0795 - acc: 0.2750 - val_loss: 0.0805 - val_acc: 0.2704
Epoch 66/100
2793/2793 [==============================] - 0s 95us/step - loss: 0.0796 - acc: 0.2689 - val_loss: 0.0802 - val_acc: 0.2704
Epoch 67/100
2793/2793 [=============================

### Compare and choosing

#### We can easily see that three models have very low accuracy on both of validation and training test because of lacking data
#### It is very easy to see that the first model is very basic and easy to build but it might be underfitting
#### Second model works quite fast and well. The accuracy of training and accuracy of testing are getting closer
#### In third model, I added some big steps which are claimed to get better result but It seems to me not affect to our model and even make model run slower
#### As the result, The second model is the best choice